## 코랩 Unsloth 설치 부분

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm
# Install latest Hugging Face for Gemma-3!
!pip install --no-deps git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

In [ ]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

## 실제 코드

In [ ]:
!pip install langchain_chroma -q
!pip install langchain_openai -q
!pip install langchain_cohere -q

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import re
import os
import torch
from datasets import load_dataset

from unsloth import FastModel,FastLanguageModel
from unsloth.chat_templates import get_chat_template, standardize_data_formats, train_on_responses_only

from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, TrainerCallback, pipeline
from huggingface_hub import login, upload_folder, create_repo

from langchain.llms import HuggingFacePipeline


from langchain_core.prompts import PromptTemplate
from langchain_chroma import Chroma
from langchain_cohere import CohereRerank
from langchain.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.retrievers import ContextualCompressionRetriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_openai import OpenAIEmbeddings # langchain과 연동된 openai에서 지원하는 pre-trained Embedding 모델
from trl import SFTTrainer, SFTConfig

# 엑세스 토큰
from google.colab import userdata

hf_token = userdata.get('hf_token')
ch_token = userdata.get('cohere')
open_token = userdata.get('openai')
login(hf_token)

In [ ]:
fourbit_models = [
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",
]

# 모델, 토크나이저 불러오기
model_12b, tokenizer_12b = FastModel.from_pretrained(
    model_name = fourbit_models[2],
    max_seq_length = 4096,   # 모델이 처리할 수 있는 시퀀스 길이
    load_in_4bit = True,     # 4비트 양자화로 메모리 줄이기
    load_in_8bit = False,    # 8비트 양자화로 메모리는 4비트의 2배
    full_finetuning = False, # 풀파인튜닝 여부 LoRA를 사용할 거면 False
)

==((====))==  Unsloth 2025.3.19: Fast Gemma3 patching. Transformers: 4.50.0.dev0. vLLM: 0.8.3.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/258k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/2.84G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

In [ ]:
#  get_peft_model : Unsloth의 LoRA 설정 함수
model_12b = FastModel.get_peft_model(
    model_12b,
    finetune_vision_layers     = False, # 시각 기반인가? False
    finetune_language_layers   = True,  # 언어 기반인가? True
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # MLP (Multi-Layer Perceptron) 파인튜닝에 중요한 역할 항상 True 유지
    # target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
    #                 "gate_proj", "up_proj", "down_proj","embed_tokens",],
    #                 # "embed_tokens", "lm_head",], # 포함하면 파라미터 수 폭증, 메모리 영향 있음

    r = 8,            # LoRA 어댑터의 랭크(차원), 높을수록 모델의 설명력이 좋아지지만 과적합 됨(일반적으로 8이나 16)
    lora_alpha = 8,   # LoRA 어댑터의 스케일링(업데이트 크기), 보통 r과 같은값(최소가 r값)
    lora_dropout = 0, # LoRA 어댑터의 드롭아웃 비율(과적합 방지) 0이나 0.1
    bias = "none",    # LoRA어댑터의 편향 추가여부 "none", "lora_only"(lora에만 추가), "all" 모두 추가
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.language_model.model` require gradients


In [ ]:
model_12b.print_trainable_parameters()

trainable params: 32,735,232 || all params: 12,220,060,272 || trainable%: 0.2679


In [ ]:
def model_response(model,tokenizer, question= "갈현동에서 카페 창업을 하려고해. 보증금 5000만에 월세 200만 원 이하로 권리금은 없었으면 좋겠고 1층에  50 m² 이상으로 찾아줘"):
    # 메시지 정의
    messages = [{
        "role": "user",
        "content": [{"type": "text", "text": question}]
    }]

    # Chat 템플릿 적용
    text = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=False
    )
    text_streamer = TextStreamer(tokenizer)
    outputs = model.generate(
        **tokenizer([text], return_tensors = "pt").to("cuda"),
        max_new_tokens = 500,
        temperature = 1.0, top_p = 0.95, top_k = 64,
    )

    response = tokenizer.batch_decode(outputs)
    parts = response[0].split("<start_of_turn>")

    user_text = parts[1].replace("user", "").replace("<end_of_turn>", "").strip()
    model_text = parts[2].replace("model", "").strip()

    print(f"""질문 : \n {user_text}""")
    print(f"""답변 : \n {model_text}""")

    return response

In [ ]:
# 기본 모델
response = model_response(model_12b, tokenizer_12b)

질문 : 
 갈현동에서 카페 창업을 하려고해. 보증금 5000만에 월세 200만 원 이하로 권리금은 없었으면 좋겠고 1층에  50 m² 이상으로 찾아줘
답변 : 
 ## 갈현동 카페 창업 가능 매물 탐색 (보증금 5000만원 이하, 월세 200만원 이하, 1층, 50m² 이상, 권리금 없음)
['<bos><start_of_turn>user\n갈현동에서 카페 창업을 하려고해. 보증금 5000만에 월세 200만 원 이하로 권리금은 없었으면 좋겠고 1층에  50 m² 이상으로 찾아줘<end_of_turn>\n<start_of_turn>model\n## 갈현동 카페 창업 가능 매물 탐색 (보증금 5000만원 이하, 월세 200만원 이하, 1층, 50m² 이상, 권리금 없음)\n\n갈현동에서 카페 창업에 필요한 조건에 맞는 매물을 찾는 것은 쉽지 않지만, 꾸준히 탐색하면 가능성이 있습니다. 현재 부동산 시장 상황과 갈현동의 특성을 고려하여 현실적인 접근 방식을 제안합니다.\n\n**1. 현실적인 기대 수준:**\n\n*   **권리금 없는 매물:** 갈현동은 비교적 임대 시세가 높은 지역이므로, 권리금 없는 매물을 찾기는 매우 어렵습니다. 장기 임대 계약 또는 건물주의 특별한 의지가 있는 경우에 가능할 수 있습니다.\n*   **50m² 이상 면적:** 50m² 이상의 1층 매물은 월세가 높아질 가능성이 높습니다. 따라서 보증금 5000만원 이하, 월세 200만원 이하 조건을 모두 만족시키기 어려울 수 있습니다.\n*   **월세 200만원 이하:** 갈현동의 상권 상황을 고려했을 때, 50m² 이상의 1층 매물을 이 가격대로 찾기는 쉽지 않습니다.\n\n**2. 탐색 방법:**\n\n*   **부동산 중개업소 방문:** 갈현동 지역의 부동산 중개업소를 방문하여 원하는 조건에 맞는 매물을 문의하는 것이 가장 일반적인 방법입니다. 여러 곳을 방문하여 정보를 얻고 비교하는 것이 좋습니다.\n*   **온라인 부동산 플랫폼 활용:**\n    *   **직방, 다방, 네이버

In [ ]:
# 데이터 형식을 표준 형식으로 변환
dataset = load_dataset("json", data_files="/content/drive/MyDrive/Colab Notebooks/data/ft_prompt.jsonl", split='train')
dataset = standardize_data_formats(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# 데이터프레임에 text열을 추가해서 gemma3에 맞는 데이터 형식 추가
def apply_chat_template(examples):
    texts = tokenizer_12b.apply_chat_template(examples["messages"]) # 토크나이저 형식대로 바꿔라
    return { "text" : texts }
pass
dataset = dataset.map(apply_chat_template, batched = True) # 함수 삽입

Map:   0%|          | 0/108 [00:00<?, ? examples/s]

In [ ]:
class CustomCheckpointCallback(TrainerCallback):
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def on_epoch_end(self, args, state, control, **kwargs):
        current_epoch = int(state.epoch)
        if current_epoch in [50, 60, 70]:
            output_dir = f"/content/drive/MyDrive/Colab Notebooks/checkpoint/checkpoint-epoch-{current_epoch}"
            kwargs["model"].save_pretrained(output_dir)
            self.tokenizer.save_pretrained(output_dir)  # 여기서 직접 저장
            print(f">>> Checkpoint saved at epoch {current_epoch}")
        return control


In [ ]:
trainer_12b_70 = SFTTrainer(
    model = model_12b,
    tokenizer = tokenizer_12b,
    train_dataset = dataset,
    eval_dataset = None,
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 70,
        learning_rate = 2e-4,
        logging_steps = 100,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none",
        output_dir="/content/drive/MyDrive/Colab Notebooks/checkpoint",
        # save_strategy="epoch",
        # save_steps= 25,
        # save_total_limit=3,
    ),

    callbacks=[CustomCheckpointCallback(tokenizer_12b)],
)

In [ ]:
# user input은 학습하지 않고 assistant output만 학습하기 위해서 해당 부분 지정
trainer_12b_70 = train_on_responses_only(
    trainer_12b_70,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

In [ ]:
trainer_12b_70.train(resume_from_checkpoint="/content/drive/MyDrive/Colab Notebooks/checkpoint/checkpoint-500")

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 108 | Num Epochs = 70 | Total steps = 910
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 32,735,232/12,000,000,000 (0.27% trained)


Step,Training Loss
600,0.012700
700,0.012600
800,0.012500
900,0.012200


>>> Checkpoint saved at epoch 50
>>> Checkpoint saved at epoch 60


TrainOutput(global_step=910, training_loss=0.005619976440301308, metrics={'train_runtime': 1973.6484, 'train_samples_per_second': 3.83, 'train_steps_per_second': 0.461, 'total_flos': 3.7063813545624384e+17, 'train_loss': 0.005619976440301308})

In [ ]:
model_12b_70 = trainer_12b_70.model
response_70 = model_response(model_12b_70, tokenizer_12b)

질문 : 
 갈현동에서 카페 창업을 하려고해. 보증금 5000만에 월세 200만 원 이하로 권리금은 없었으면 좋겠고 1층에  50 m² 이상으로 찾아줘
답변 : 
 갈현동에서 카페 창업에 적합한 매물을 찾는다면, 다음과 같은 정보를 알려드릴게요.
['<bos><start_of_turn>user\n갈현동에서 카페 창업을 하려고해. 보증금 5000만에 월세 200만 원 이하로 권리금은 없었으면 좋겠고 1층에  50 m² 이상으로 찾아줘<end_of_turn>\n<start_of_turn>model\n갈현동에서 카페 창업에 적합한 매물을 찾는다면, 다음과 같은 정보를 알려드릴게요.\n\n현재 갈현동에는 월세 200만 원 이하, 보증금 5000만 원 이하인 카페 매물이 2개나 있어요!\n\n**매물 1**\n\n*   보증금: 2000만 원\n*   월세: 170만 원\n*   권리금: 없음\n*   면적: 56.2 m²\n*   특징: 넓은 공간과 좋은 입지로 다양한 카페 운영이 가능해요.\n\n**매물 2**\n\n*   보증금: 3000만 원\n*   월세: 165만 원\n*   권리금: 없음\n*   면적: 66.1 m²\n*   특징: 비교적 넓은 공간과 합리적인 월세로 안정적인 카페 운영이 가능해요.\n\n더 자세한 정보나 추가적인 매물 추천이 필요하시면 말씀해주세요! 😊<end_of_turn>']


**모델을 로컬에 저장**

In [ ]:
local_id = 'test'
model_12b_70.save_pretrained_merged(local_id,tokenizer_12b)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

model.safetensors.index.json:   0%|          | 0.00/109k [00:00<?, ?B/s]

Unsloth: Merging weights into 16bit:   0%|          | 0/5 [00:00<?, ?it/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Unsloth: Merging weights into 16bit:  20%|██        | 1/5 [00:42<02:50, 42.57s/it]

...

Unsloth: Merging weights into 16bit:  60%|██████    | 3/5 [02:11<01:28, 44.04s/it]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

Unsloth: Merging weights into 16bit:  80%|████████  | 4/5 [02:55<00:44, 44.01s/it]

model-00005-of-00005.safetensors:   0%|          | 0.00/4.60G [00:00<?, ?B/s]

Unsloth: Merging weights into 16bit: 100%|██████████| 5/5 [03:51<00:00, 46.24s/it]


**허깅페이스에 모델 업로드**

In [ ]:
repo_id = "Envy1025/Retail_Property_Finder_Chatbot"

# 레포지토리 생성
create_repo(
    repo_id=repo_id,
    repo_type="model",
    private=False  # True로 하면 비공개 레포로 생성됨
)

# 생성한 레포지토리에 로컬에 저장된 모델 폴더 업로드
upload_folder(
    folder_path=local_id,
    repo_id=repo_id,
)

In [ ]:
def stream_response(model,tokenizer, question= "갈현동에서 카페 창업을 하려고해. 보증금 5000만에 월세 200만 원 이하로 권리금은 없었으면 좋겠고 1층에  50 m² 이상으로 찾아줘"):
    messages = [{
        "role": "user",
        "content": [{"type": "text", "text": question}]
    }]
    # FastLanguageModel.for_inference(model) # Enable native 2x faster inference
    inputs = tokenizer.apply_chat_template(messages, tokenize = True, add_generation_prompt = True, return_tensors = "pt").to("cuda")

    text_streamer = TextStreamer(tokenizer)
    outputs = model.generate(input_ids = inputs,
                             streamer = text_streamer,
                             max_new_tokens = 500,
                             use_cache = True,
                             temperature = 1.0, top_p = 0.95, top_k = 64,)

    response = tokenizer.batch_decode(outputs)
    parts = response[0].split("<start_of_turn>")

    user_text = parts[1].replace("user", "").replace("<end_of_turn>", "").strip()
    model_text = parts[2].replace("model", "").strip()

    print(f"""질문 : \n {user_text}""")
    print(f"""답변 : \n {model_text}""")


    return response

In [ ]:
# @title 허깅페이스에서 모델 불러오기
repo_id = "Envy1025/Retail_Property_Finder_Chatbot"

fast_model, fast_tokenizer = FastLanguageModel.from_pretrained(
    model_name = repo_id,
    max_seq_length = 4096,
    device_map="auto",
    load_in_4bit = True,
)

In [ ]:
hg_model = AutoModelForCausalLM.from_pretrained(repo_id)
hg_tokenizer = AutoTokenizer.from_pretrained(repo_id)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Some weights of the model checkpoint at Envy1025/Retail_Property_Finder_Chatbot were not used when initializing Gemma3ForCausalLM: {'vision_tower.vision_model.encoder.layers.14.layer_norm2.bias', 'vision_tower.vision_model.encoder.layers.26.self_attn.q_proj.bias', 'vision_tower.vision_model.encoder.layers.12.self_attn.v_proj.weight', 'vision_tower.vision_model.encoder.layers.24.mlp.fc1.weight', 'vision_tower.vision_model.encoder.layers.18.mlp.fc2.bias', 'vision_tower.vision_model.encoder.layers.21.mlp.fc2.bias', 'vision_tower.vision_model.encoder.layers.14.self_attn.k_proj.bias', 'vision_tower.vision_model.embeddings.patch_embedding.bias', 'vision_tower.vision_model.encoder.layers.22.self_attn.q_proj.bias', 'vision_tower.vision_model.encoder.layers.22.self_attn.k_proj.bias', 'vision_tower.vision_model.encoder.layers.9.layer_norm2.bias', 'vision_tower.vision_model.encoder.layers.23.layer_norm1.weight', 'vision_tower.vision_model.encoder.layers.1.self_attn.v_proj.weight', 'vision_tower.v

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

In [ ]:
# hg_model.cuda()
hg_response = model_response(hg_model,hg_tokenizer)

In [ ]:
# @title 체크포인트로 불러오기
checkpoint_path = "/content/drive/MyDrive/Colab Notebooks/checkpoint/checkpoint-epoch-60"

# 모델과 토크나이저 로드
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=checkpoint_path,
    max_seq_length=4096,
    load_in_4bit=True,
)

model.eval()  # 추론 모드로 전환

==((====))==  Unsloth 2025.3.19: Fast Gemma3 patching. Transformers: 4.50.0.dev0. vLLM: 0.8.3.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Gemma3ForConditionalGeneration(
  (vision_tower): SiglipVisionModel(
    (vision_model): SiglipVisionTransformer(
      (embeddings): SiglipVisionEmbeddings(
        (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
        (position_embedding): Embedding(4096, 1152)
      )
      (encoder): SiglipEncoder(
        (layers): ModuleList(
          (0-26): 27 x SiglipEncoderLayer(
            (self_attn): SiglipSdpaAttention(
              (k_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (v_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (q_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (out_proj): Linear(in_features=1152, out_features=1152, bias=True)
            )
            (layer_norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
            (mlp): SiglipMLP(
              (activation_fn): PytorchGELUTanh()
              (fc1): Linear(in_features=1

#### 래그

In [ ]:
# 임베딩 모델 정의
my_embedding = OpenAIEmbeddings(model = "text-embedding-3-small",
                                api_key = open_token
                               )
# DB저장 경로
my_directory = "/content/drive/MyDrive/Colab Notebooks/vector"

vectordb = Chroma(persist_directory=my_directory, embedding_function=my_embedding)

In [ ]:
vectordb._collection.count()

134411

In [ ]:
# MMR검색기 지정
mmr_retriever = vectordb.as_retriever(
    search_type="mmr",
    search_kwargs={
        "fetch_k": 100,   # 후보 문서 개수를 줄여 정보 과부하 방지
        "k": 10,          # 최종 선택 문서 개수를 줄여 정밀도 개선
        "lambda_mult": 1  # 유사도를 더 우선시하도록 조정
    }
)

docs = mmr_retriever.get_relevant_documents("서교동 월세 50만원 권리금 2천만원 1층 보증금 5000만원")
docs

[Document(id='f9cca2ce-e92f-4b9a-ab69-c7e88a5039ed', metadata={}, page_content='ID: 423326\n제목: 보증금3000만원월세50만원권리금2000만원\n위치: 관악구 신림동\n업종: 치킨닭요리\n보증금: 3000만원\n월세: 50만원\n권리금: 2000만원\n면적(㎡): 29.7\n층수: 1층\n설명: 배달업추천오랫동안영업해서당골많음꾸준히매출나오는집집주인분이100070도생각하신다고외부뒷공간이용가능지하주차장따로있음'),
 Document(id='8da5ceea-fbfe-4ae1-94d6-2bc0efbb4754', metadata={}, page_content='ID: 586639\n제목: 월세50만원이하동빙고동상가\n위치: 용산구 동빙고동\n업종: 퓨전쌀국수\n보증금: 500만원\n월세: 30만원\n권리금: 200만원\n면적(㎡): 26.4\n층수: 1층\n설명: 인근에아파트,오피스텔,주택가들이많아서거주민과회사원들의수요가많습1층에위치해있어서눈에잘띄고,접근성이우수내부인테리어가잘갖추어져있어서동종업종을하시는경우초창기시간과비용을아끼실수있습네모인이직접확인하는로안심하고주세요친절하게이외에도다양한을보유하고있으니을다하겠습상단의정보는최초건축물등록기준이며변동된사항은설명란을'),
 Document(id='31f83d8e-2a44-48e2-b870-023dc7ecb2c5', metadata={}, page_content='ID: 586007\n제목: 월세50만원이하제기동공실\n위치: 동대문구 제기동\n업종: 기타\n보증금: 500만원\n월세: 35만원\n권리금: 없음\n면적(㎡): 19.8\n층수: 1층\n설명: 상권이잘형성되어있고유동인구가많아서매출이안정적1층에위치해있어서눈에잘띄고,접근성이우수인근에,오피스텔,주택가들이많아서거주민의수요가많습네모인이직접확인하는로안심하고주세요친절하게이외에도다양한을보유하고있으니을다하겠습상단의정보는최초건축물등록기준이며변동된사항은설명란을'),
 Document(id='6e44e47e-544d-46

In [ ]:
# CohereRerank 객체 설정
my_rerank = CohereRerank(cohere_api_key = ch_token,
                         model = "rerank-v3.5"     # cohere 지원 reranking 모델
                        )

In [ ]:
# 예제 프롬프트 템플릿 생성
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "해당 조건에 맞는 매물을 추천해주세요. {context}"),
        ("ai", "{output}"),
    ]
)

In [ ]:
# Few-Shot 예시 작성
# 예시 1번
examples = [
    {
        "context": """
        갈현동에서 카페 창업을 하려고 해.
        보증금 5000만 원 이하, 월세 200만 원 이하, 권리금 없음.
        1층에 20m² 이상인 매물을 찾아줘.
        """,
        "output": """
        📍 갈현동 카페 창업 추천 매물

        ───────────────────────────

        🏠 매물 1
        위치: 은평구 갈현동
        보증금: 1000 만원
        월세: 72 만원
        권리금: 100 만원
        면적: 33.1m²
        층수: 1층

        🔗 매물 보러가기: [클릭](https://www.zigbang.com/home/store/items/512818)
        """

},

# 예시 2번
{
        "context": """
        서울에서 보증금 3천 만원, 월세 80 만원 이하로 음식점을 위한 매물을 구할 수 있을까?
        고층이어도 상관없고, 10m²보다는 컸으면 좋겠어.
        """,
        "output": """
        📍 음식점 창업 매물 추천

        원하는 조건에 맞는 매물을 찾는다면, 중구는 어떤가요?

        ───────────────────────────

        🏠 매물 1
        위치: 중구 필동2가
        보증금: 1000 만원
        월세: 80 만원
        권리금: 1500 만원
        면적: 23.1m²
        층수: 1층
        설명: 상권이잘형성되어있어서유동인구가많아유입되는손님이꾸준히있어서매출이꾸준히나오는매장.
        추천 이유: 원하는 조건에 맞는 매물입니다.

        🔗 매물 보러가기: [클릭](https://www.zigbang.com/home/store/items/571420)

        ───────────────────────────

        🏠 매물 2
        위치: 중구 쌍림동
        보증금: 0 만원
        월세: 75 만원
        권리금: 6000 만원
        면적: 56.2 m²
        층수: 1층
        설명: 역초역세권카페매물
        추천 이유: 월세는 요청하신 80 만원 보다는 높지만 보증금이 없어 고려해볼만한 매물입니다.

        🔗 매물 보러가기: [클릭](https://www.zigbang.com/home/store/items/515478)
        """
},
]

In [ ]:
# Few-shot 프롬프트 템플릿 생성
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt,
    examples = examples,
)

In [ ]:
# 최종 프롬프트 템플릿 생성
final_prompt = ChatPromptTemplate(
    [
        ("system", """
당신은 상가 부동산 매물 추천 전문가입니다.
사용자 정보를 기반으로, 가장 적합한 상가 매물을 추천합니다.
문서의 ID는 vectordb의 'ID' 키 값과 일치합니다. 추천 매물에 대해 추측하거나 임의로 정보를 생성하지 마세요.

───────────────────────────

📍 요청사항
1. **위치**, **보증금**, **월세** 조건에 맞는 매물을 1개 이상 추천하세요.
2. 각 상가가 왜 적합한지에 대한 구체적이고 명확한 이유를 설명하세요.
3. 추천 매물의 링크는 문서 ID와 일치해야 하며, 잘못된 링크나 매물은 제공하지 마세요.
4. 만약 조건에 맞는 매물이 없다면, 다른 지역에서 비슷한 조건의 매물 정보를 링크까지 제공하세요.
5. 추천 이유는 사용자가 제공한 정보의 근거로만 작성해야 합니다. 추측하거나 임의로 답변하지 마세요.

"""

         ),
        few_shot_prompt,
         ("human",
     "참고 문서: {context}\n\n"
     "질문: {input}\n\n"
     "제공된 문서들을 참고하여 신뢰할 수 있는 답변을 한글로 작성하세요. "),
    ]
)

In [ ]:
question= "갈현동에서 카페 창업을 하려고해. 보증금 5000만에 월세 200만 원 이하로 권리금은 없었으면 좋겠고 1층에  50 m² 이상으로 찾아줘"

In [ ]:
# LLM 파이프라인 구성
hf_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    task="text-generation",
    max_new_tokens=512,
    temperature=0.7,
    do_sample=True,
)

llm = HuggingFacePipeline(pipeline=hf_pipeline)


Device set to use cuda:0
The model 'Gemma3ForConditionalGeneration' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForCausalLM', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapan

In [ ]:
# (1) CohereRerank + MMR 검색기
compression_retriever = ContextualCompressionRetriever(base_compressor = my_rerank,
                                                       base_retriever = mmr_retriever
                                                      )
# (2) LLM + Few-shot
combine_docs_chain = create_stuff_documents_chain(llm,
                                                  final_prompt
                                                  )
# (1) + (2)
retrieval_rerank_chain = create_retrieval_chain(compression_retriever,
                                         combine_docs_chain)

# question 질문
mmr_response = retrieval_rerank_chain.invoke({"input": question})
# print(f'질문 : \n{mmr_response.get("input")}')
# print()
# print(f'''답변 : \n{mmr_response.get("answer")}''')

# print()
# print('-'*50)
# print()

# print(f'참고문서1 : {mmr_response.get("context")[0].page_content}')
# print()
# print(f'참고문서2 : {mmr_response.get("context")[1].page_content}')
# print()
# print(f'참고문서3 : {mmr_response.get("context")[2].page_content}')

In [ ]:
print(mmr_response['answer'][mmr_response['answer'].rfind("Human:"):])

Human: 참고 문서: 젊은 층이 많은 지역

젊은 층이 많은 지역

유동 인구가 많은 지역

질문: 갈현동에서 카페 창업을 하려고해. 보증금 5000만에 월세 200만 원 이하로 권리금은 없었으면 좋겠고 1층에  50 m² 이상으로 찾아줘

제공된 문서들을 참고하여 신뢰할 수 있는 답변을 한글로 작성하세요. 
AI: 
        📍 갈현동 카페 창업 추천 매물

        ───────────────────────────

        🏠 매물 1
        위치: 은평구 갈현동
        보증금: 1000 만원
        월세: 72 만원
        권리금: 100 만원
        면적: 33.1m²
        층수: 1층

        🔗 매물 보러가기: [클릭](https://www.zigbang.com/home/store/items/512818)

        추천 이유: 갈현동은 젊은 층이 많은 지역으로, 카페 창업에 적합한 위치입니다. 보증금 1000만 원, 월세 72만 원으로 비교적 저렴한 가격에 매물을 구할 수 있습니다. 다만, 면적은 33.1m²로 요청하신 50m²보다는 작습니다. 유동 인구가 많은 지역인지 확인이 필요합니다.



In [ ]:
mmr_response2 = retrieval_rerank_chain.invoke({"input": '쌍문동에서 보증금 3000만 원 이하, 월세 100만원 이하인 곳있을까'})
print(mmr_response2['answer'][mmr_response2['answer'].rfind("Human:"):])

Human: 참고 문서: 젊은 층이 많은 지역

젊은 층이 많은 지역

유동 인구가 많은 지역

질문: 쌍문동에서 보증금 3000만 원 이하, 월세 100만원 이하인 곳있을까

제공된 문서들을 참고하여 신뢰할 수 있는 답변을 한글로 작성하세요. 
AI: 
        📍 쌍문동 매물 추천

        쌍문동은 젊은 층이 많은 지역으로 유동 인구가 많아, 보증금 3000만 원 이하, 월세 100만원 이하의 매물을 찾아보았습니다.

        ───────────────────────────

        🏠 매물 1
        위치: 강북구 쌍문동
        보증금: 2000 만원
        월세: 70 만원
        권리금: 2500 만원
        면적: 27.7m²
        층수: 1층

        🔗 매물 보러가기: [클릭](https://www.zigbang.com/home/store/items/511707)

        추천 이유: 젊은 층이 많은 지역이며, 유동 인구가 많은 쌍문동에 위치해 있습니다. 보증금과 월세 조건도 충족합니다.
        
        ───────────────────────────

        🏠 매물 2
        위치: 강북구 쌍문동
        보증금: 3000 만원
        월세: 90 만원
        권리금: 1000 만원
        면적: 16.7m²
        층수: 1층

        🔗 매물 보러가기: [클릭](https://www.zigbang.com/home/store/items/511708)

        추천 이유: 젊은 층이 많은 지역이며, 유동 인구가 많은 쌍문동에 위치해 있습니다. 보증금과 월세 조건도 충족합니다.
        
        ───────────────────────────
        
        🏠 매물 3
        위치: 강북구 쌍문동
        보증금: 3000 만원
      